<center><img src="https://www.inteli.edu.br/wp-content/uploads/2021/08/20172028/marca_1-2.png" width="50%" height="50%"/></center>

<h1 align='center'><b>IA para Marketing: Monitoramento de campanhas utilizando processamento de linguagem natural (PLN)<b></h1>

<center><img src="https://upload.wikimedia.org/wikipedia/commons/c/c2/Btg-logo-blue.svg" width="50%" height="50%"/></center>

<h2 align='center'>O Banco BTG Pactual enfrenta um desafio na área de Marketing em entender as necessidades e demandas dos clientes de maneira fácil e rápida nas redes sociais. A solução proposta para esse problema foi o desenvolvimento de uma Inteligência Artificial utilizando processamento de linguagem natural (PLN), capaz de monitorar as campanhas de marketing, voltadas para o Instagram. O objetivo principal dessa solução é rastrear os dados em tempo real, analisar e interpretar as mensagens e comentários enviados pelos clientes na rede social, a fim de identificar as necessidades e demandas de forma precisa e eficiente.</h2>

---

# Sobre os dados

Esse projeto estará utilizando dados coletados e tratados pela equipe de Automation do BTG Pactual, o qual disponibilizou o dataset. Com base nas informações dispostas nesse dataset, realizaremos insights a cerca dos comentários feitos nos posts do Instagram do próprio banco. Vale lembrar que os dados estão anonimizados e resguardados para manter a privacidade e ética com os usuários e com o banco.

# Montagem do Drive (utilização do Google Colab)

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

# **Instalação de Bibliotecas - Setup**

In [ ]:
import pandas as pd
import numpy as np

import nltk
nltk.download('punkt')
nltk.download('stopwords')

import re

from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
import plotly.express as px

import random

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Loading Data
Fiz algumas alterações em algumas linhas do dataset, devido o encontro de alguns erros. Além disso, fiz a leitura em arquivo XLSX. pois estava dando erro na leitura do arquivo em csv. 

In [ ]:
df = pd.read_excel('/content/drive/MyDrive/Módulo 6/projeto/base_consertada_excel.xlsx')
df

,Column1,id,autor,texto,shortCode,tipoInteracao,tipoMidia,idInteracao,idPai,URL,dataPublicada,dataInsercao,anomalia,probabilidadeAnomalia,processado,contemHyperlink
0,0,1,btgpactual,"Olá, Ana Paula! Como vai?\n\nSentimos muito pe...",CkG9KXjpY2B,reply,REELS,17949834317264462,17988010663613927,https://www.instagram.com/p/CkG9KXjpY2B/c/1798...,2022-10-25 15:35:24,2022-10-25 21:52:54,-1,0,0,-1
1,1,2,btgpactual,"Olá, Rosana! Tudo bem?\n\nPara te orientarmos ...",CkG9KXjpY2B,reply,REELS,17968362370910874,17955765434145877,https://www.instagram.com/p/CkG9KXjpY2B/c/1795...,2022-10-25 12:08:06,2022-10-25 21:52:54,-1,0,0,-1
2,2,3,btgpactual,@luizantoniojovinodossan 🚀 💙,CkG9KXjpY2B,reply,REELS,17968676113804345,17868806699792839,https://www.instagram.com/p/CkG9KXjpY2B/c/1786...,2022-10-25 08:16:15,2022-10-25 21:52:54,-1,0,0,-1
3,3,4,btgpactual,@ronialencar Boa tarde! Tudo bem?\n\nLamentamo...,CkG9KXjpY2B,reply,REELS,18045922528372377,18327741553009776,https://www.instagram.com/p/CkG9KXjpY2B/c/1832...,2022-10-25 09:39:32,2022-10-25 21:52:55,-1,0,0,-1
4,4,5,btgpactual,"Oi, Nelson! Tudo bem?\n\nPossuímos soluções co...",CkG9KXjpY2B,reply,REELS,17864851241797946,18159475801266342,https://www.instagram.com/p/CkG9KXjpY2B/c/1815...,2022-10-25 09:38:38,2022-10-25 21:52:55,-1,0,0,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4388,4544,3028308,caioaugustofr,"Estão contratando muitos blogueiros, mas eu fi...",Cn4bcg7LM4t,comment,REELS,17965800242039792,-,https://www.instagram.com/p/Cn4bcg7LM4t/c/1796...,2023-01-26 18:41:35,2023-01-26 19:01:29,-1,0,0,-1
4389,4545,3028501,btgpactual,@xubs._ 💙,Cn4bcg7LM4t,reply,REELS,18340331500020596,18014457793513467,https://www.instagram.com/p/Cn4bcg7LM4t/c/1801...,2023-01-26 18:44:27,2023-01-26 19:02:16,-1,0,0,-1
4390,4546,3028580,btgpactual,Agradecemos o seu feedback! 💙,CnzSr6ltH74,reply,REELS,18004216693521888,17957000375356996,https://www.instagram.com/p/CnzSr6ltH74/c/1795...,2023-01-26 18:45:13,2023-01-26 19:02:35,-1,0,0,-1
4391,4547,3029097,je_mascarenhas,Arrasa Micaaa 💙✨🙏🏼👏🏼👏🏼👏🏼👏🏼,Cn4bcg7LM4t,comment,REELS,17969884580114389,-,https://www.instagram.com/p/Cn4bcg7LM4t/c/1796...,2023-01-26 19:26:41,2023-01-26 19:31:45,-1,0,0,-1


In [ ]:
df.shape

(4393, 16)

In [ ]:
df['probabilidadeAnomalia'].value_counts() # confirmando se há algum erro no dataset, pois havia erros nessa coluna em específico
# Entender qual a função dessa coluna!

0    4393
Name: probabilidadeAnomalia, dtype: int64

In [ ]:
df['processado'].value_counts() # Entender qual a função dessa coluna!

0    4393
Name: processado, dtype: int64

In [ ]:
df['contemHyperlink'].value_counts() # Entender qual a função dessa coluna!

-1    4393
Name: contemHyperlink, dtype: int64

In [ ]:
df['anomalia'].value_counts() # Entender qual a função dessa coluna!

-1    4393
Name: anomalia, dtype: int64

# **Exploratória** dos dados

In [ ]:
# removendo colunas que não serão utilizadas
df.drop(['Column1', 'URL','anomalia', 'probabilidadeAnomalia', 'processado', 'contemHyperlink'], axis=1, inplace = True) # estarei dropando a coluna 'column1' devido as alterações no dataset, a indexação da coluna está errado, então seguirei com a indexação padrão
df.head()

,id,autor,texto,shortCode,tipoInteracao,tipoMidia,idInteracao,idPai,dataPublicada,dataInsercao
0,1,btgpactual,"Olá, Ana Paula! Como vai?\n\nSentimos muito pe...",CkG9KXjpY2B,reply,REELS,17949834317264462,17988010663613927,2022-10-25 15:35:24,2022-10-25 21:52:54
1,2,btgpactual,"Olá, Rosana! Tudo bem?\n\nPara te orientarmos ...",CkG9KXjpY2B,reply,REELS,17968362370910874,17955765434145877,2022-10-25 12:08:06,2022-10-25 21:52:54
2,3,btgpactual,@luizantoniojovinodossan 🚀 💙,CkG9KXjpY2B,reply,REELS,17968676113804345,17868806699792839,2022-10-25 08:16:15,2022-10-25 21:52:54
3,4,btgpactual,@ronialencar Boa tarde! Tudo bem?\n\nLamentamo...,CkG9KXjpY2B,reply,REELS,18045922528372377,18327741553009776,2022-10-25 09:39:32,2022-10-25 21:52:55
4,5,btgpactual,"Oi, Nelson! Tudo bem?\n\nPossuímos soluções co...",CkG9KXjpY2B,reply,REELS,17864851241797946,18159475801266342,2022-10-25 09:38:38,2022-10-25 21:52:55


In [ ]:
df['tipoInteracao'].value_counts()

comment    2585
mention     600
reply       316
Name: tipoInteracao, dtype: int64

In [ ]:
tipo_interacao_counts = df['tipoInteracao'].value_counts()

fig = px.bar(tipo_interacao_counts, x=tipo_interacao_counts.index, y='tipoInteracao',
             labels={'tipoInteracao': 'Contagem', 'x': 'Tipo de Interação'},
             title='Distribuição dos tipos de Interações')
fig.show()

In [ ]:
df['tipoMidia'].value_counts()

REELS             1525
FEED              1376
IMAGE              215
VIDEO              201
CAROUSEL_ALBUM     184
Name: tipoMidia, dtype: int64

In [ ]:
# LEMBRAR: tentar distribuir os REELS entre os que estão dentro e fora do FEED
tipo_midia_counts = df['tipoMidia'].value_counts()

fig = px.bar(tipo_midia_counts, x=tipo_midia_counts.index, y='tipoMidia',
             labels={'tipoMidia': 'Contagem', 'x': 'Tipo de Mídia'},
             title='Distribuição dos tipos de Mídias do Instagram')
fig.show()

In [ ]:
df['shortCode'].value_counts()

CnzSr6ltH74    219
CnxFYt8pj-c    181
Cn4bcg7LM4t    138
CnwdMFtp0mu    112
Clom68VJcSX     85
              ... 
ClG1JWhAtQp      1
ClG2U2qOpJQ      1
ClHB6sXDaSq      1
ClHEs6Qu30n      1
Cn5MCVvP1H3      1
Name: shortCode, Length: 703, dtype: int64

In [ ]:
# retirando toda e qualquer informação provinda do btgpactual para fins de análise somente dos usuários
df = df.loc[df['autor'] != 'btgpactual']
df

,id,autor,texto,shortCode,tipoInteracao,tipoMidia,idInteracao,idPai,dataPublicada,dataInsercao
10,11,matheusriosortodontia,@gustavopaulinno Realmente. Ja rodei por aí e ...,CkG9KXjpY2B,reply,REELS,17961352247055161,17983688287717831,2022-10-24 17:40:49,2022-10-25 21:52:56
13,14,erikacoimbra,👏👏👏,CkG9KXjpY2B,comment,REELS,17943066782219685,-,2022-10-25 19:13:45,2022-10-25 21:52:57
14,15,anapaulanakade,"O app está travando desde ontem, está em manut...",CkG9KXjpY2B,comment,REELS,17988010663613927,-,2022-10-25 12:54:54,2022-10-25 21:52:57
15,16,rosanakassar,"Abri uma conta, mas achei muito complicado inv...",CkG9KXjpY2B,comment,REELS,17955765434145877,-,2022-10-25 12:03:51,2022-10-25 21:52:58
16,17,luizantoniojovinodossan,👏,CkG9KXjpY2B,comment,REELS,17868806699792839,-,2022-10-25 00:30:31,2022-10-25 21:52:58
...,...,...,...,...,...,...,...,...,...,...
4386,3028275,gui.decicco,Participe da conferência econômica mais import...,Cn5MCVvP1H3,mention,CAROUSEL_ALBUM,17989315081750052,-,2023-01-26 18:31:44,2023-01-26 18:35:02
4387,3028307,giballiana,A melhor que temos 🥰🥰,Cn4bcg7LM4t,comment,REELS,17913917285610958,-,2023-01-26 18:47:19,2023-01-26 19:01:29
4388,3028308,caioaugustofr,"Estão contratando muitos blogueiros, mas eu fi...",Cn4bcg7LM4t,comment,REELS,17965800242039792,-,2023-01-26 18:41:35,2023-01-26 19:01:29
4391,3029097,je_mascarenhas,Arrasa Micaaa 💙✨🙏🏼👏🏼👏🏼👏🏼👏🏼,Cn4bcg7LM4t,comment,REELS,17969884580114389,-,2023-01-26 19:26:41,2023-01-26 19:31:45


## Descrevendo as features:
- id: provável id que foi salvo aquela informação
- autor: proprietário da mensagem
- texto: conteúdo da mensagem
- shortCode: código referente ao determinado post (CONFIRMAR)
- tipoInteracao: modo que foi enviada a mensagem, um simples comentário ou uma resposta a um comentário, sendo os seguintes tipos e suas respectivas quantias:
  - **comment** - 2585
  - **reply** - 1208
  - **mention** - 600

- tipoMidia: meios em que os conteúdos apareceram e suas respectivas quantias
  - **REELS** - 2016 (total de reels existentes)
    - REELS *fora do feed* - 839
  - **FEED** - 1777 (todo tipo de conteúdo no feed)
    - REELS *no feed* - 1177
    - IMAGE - 215 (imagens, que vão para o FEED)
    - VIDEO - 201 (vídeos, que vão para o FEED)
    - CAROUSEL_ALBUM - 184 (conjunto de fotos, que vão para o FEED)
- idInteracao: id daquela interação (CONFIRMAR)
- idPai: ? (CONFIRMAR)
- dataPublicada: data de publicação da interação (CONFIRMAR)
- dataInsercao: possível data de inserção da interação ao dataset (CONFIRMAR)

In [ ]:
df.dtypes

id                int64
autor            object
texto            object
shortCode        object
tipoInteracao    object
tipoMidia        object
idInteracao       int64
idPai            object
dataPublicada    object
dataInsercao     object
dtype: object

# Bag of Words (BoW)

- Coluna de conteúdo que iremos tratar e fazer a normalização textual

In [ ]:
df['texto'] # Os seguintes textos não possuem o 'btgpactual' como autor.

10      @gustavopaulinno Realmente. Ja rodei por aí e ...
13                                                    👏👏👏
14      O app está travando desde ontem, está em manut...
15      Abri uma conta, mas achei muito complicado inv...
16                                                      👏
                              ...                        
4386    Participe da conferência econômica mais import...
4387                                A melhor que temos 🥰🥰
4388    Estão contratando muitos blogueiros, mas eu fi...
4391                           Arrasa Micaaa 💙✨🙏🏼👏🏼👏🏼👏🏼👏🏼
4392                                        Excelência! 👏
Name: texto, Length: 3501, dtype: object

## Definição das funções

Tokenização + Retirada de Stopwords + Remoção de caracteres especiais

In [ ]:
# definindo a função teste que recebe como parâmetro uma lista de frases e retorna uma lista de palavras sem as stopwords, caracteres especiais e tokenizada
def tokenize(frases):
  # definindo a lista que será retornada
  lista = []

  # importando o módulo nltk para a remoção das stopwords
  stopwords = nltk.corpus.stopwords.words('portuguese')

  # loop para percorrer a lista de frases
  for frase in frases:
    # removendo os caracteres especiais
    frasezinha = re.sub((r'[^\w\s]'), ' ', frase)

    # tokenizando a frase e alterando para minúsculo
    termos = nltk.word_tokenize(frasezinha.lower())

    # loop para percorrer a lista de termos
    for palavra in termos:

      # verificando se a palavra não está na lista de stopwords, se estiver não adiciona na lista  
      if palavra not in stopwords:

        # adicionando a palavra na lista
        lista.append(palavra)
        
  # retornando a lista
  return lista

In [ ]:
# Função de geração de BOW
def gerando_bow(frases):
  vocab = tokenize(frases) # Função que gera o vocabulário a partir das frases já tratadas
  print(f"Lista de palavras das frases em português \n{vocab} \n") # Imprime o vocabulário
  
  for frase in frases: # Loop para gerar o BOW

    bag_vetores = np.zeros(len(vocab)) # Cria um vetor de zeros com o tamanho do vocabulário

    palavras = nltk.word_tokenize(frase.lower()) # Tokeniza as palavras da frase

    for w in palavras: # Loop para contar as palavras

      for i, palavra in enumerate(vocab): # Loop para percorrer o vocabulário

        if palavra == w: # Se a palavra for igual a palavra do vocabulário

          bag_vetores[i] += 1 # Adiciona 1 ao vetor
          
    print(f"{frase} \n{bag_vetores}\n") # Imprime a frase e o vetor

## Teste das funções

In [ ]:
gerando_bow(df['texto'])

A saída de streaming foi truncada nas últimas 5000 linhas.
Ontem foi um dia especial, com a palestra da filósofa, @djamilaribeiro1 , uma verdadeira aula de diversidade e inclusão, promovido pelo Rio Open, que também anunciou todas seus projetos ambientais e sociais para 2023. 

Uma das novidades para 2023 será a implementação de uma ouvidoria, para atender e oferecer a melhor experiência sempre ao seu público. 

Obrigado @rioopenoficial , vocês fazem a diferença.

@marcia.casz @gmgaertner20 @marianomferraz @otavio_vilarinho @fabricioganem @bvalladao 

#teamfuturobom #djamilaribeiro #rioopen #brasil #teatro 
[0. 0. 0. ... 0. 0. 0.]

@neide1novais vamos marcar de ir 
[0. 0. 0. ... 0. 0. 0.]

Hoje recebi um excelente atendimento pelo assessor Gui Miyasaki, na qual montamos um carteira 🚀 
[0. 0. 0. ... 0. 0. 0.]

Eu queria muito entender o porquê tiraram meu limite sem dar explicação nenhuma, sempre paguei as faturas em dias e agora meu limite está baixíssimo, já estou pensando em sair des

In [ ]:
variavel = tokenize(df['texto'])

In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(variavel)

In [ ]:
tokenizer.word_counts

OrderedDict([('gustavopaulinno', 1),
             ('realmente', 56),
             ('ja', 11),
             ('rodei', 1),
             ('aí', 48),
             ('observei', 1),
             ('q', 37),
             ('quesito', 1),
             ('investimentos', 182),
             ('btgpactual', 346),
             ('número', 9),
             ('app', 53),
             ('travando', 1),
             ('desde', 56),
             ('ontem', 19),
             ('manutenção', 10),
             ('abri', 20),
             ('conta', 286),
             ('achei', 14),
             ('complicado', 5),
             ('investir', 89),
             ('renda', 44),
             ('variável', 7),
             ('confuso', 1),
             ('sai', 2),
             ('prezados', 2),
             ('btg', 488),
             ('banking', 9),
             ('ar', 4),
             ('dia', 166),
             ('todo', 97),
             ('dificuldando', 1),
             ('vida', 100),
             ('oq', 3),
             ('aco